In [ ]:
import pandas as pd

# 1. unsmile에서 혐오 컬럼 통합, 악플/욕설과 clean 그대로 유지
unsmile_path = "/mnt/data/unsmile_train_v1.0.tsv"
unsmile_df = pd.read_csv(unsmile_path, sep='\t')

hate_columns = ["여성/가족", "남성", "성소수자", "인종/국적", "연령", "지역", "종교", "기타 혐오"]
unsmile_df["혐오"] = unsmile_df[hate_columns].sum(axis=1).clip(upper=1)

# 필요한 컬럼만 유지
unsmile_df = unsmile_df[["문장", "혐오", "악플/욕설", "clean"]]
unsmile_df["학교폭력"] = 0

# 2. 학교폭력 데이터 불러오기
school_minor_path = "/mnt/data/dcinside_학교폭력마이너갤러리_데이터정제.xlsx"
school_mini_path = "/mnt/data/dcinside_학교폭력미니갤러리_데이터정제.xlsx"
school_reply_path = "/mnt/data/dcinside_응답하라학교폭력갤러리_데이터정제.xlsx"

school_minor_df = pd.read_excel(school_minor_path)
school_mini_df = pd.read_excel(school_mini_path)
school_reply_df = pd.read_excel(school_reply_path)

# 3. 내용 컬럼 병합
school_texts = pd.concat([
    school_minor_df["내용"],
    school_mini_df["내용"],
    school_reply_df["내용"]
], ignore_index=True).dropna().drop_duplicates().reset_index(drop=True)

# 4. 학교폭력 데이터프레임 생성
school_df = pd.DataFrame({
    "문장": school_texts,
    "혐오": 0,
    "악플/욕설": 0,
    "clean": 0,
    "학교폭력": 1
})

# 5. 혐오 샘플링
hate_df = unsmile_df[unsmile_df["혐오"] == 1]
non_hate_df = unsmile_df[unsmile_df["혐오"] == 0]

hate_df_sampled = hate_df.sample(n=5000, random_state=42) if len(hate_df) > 5000 else hate_df

# 6. 최종 병합 및 셔플
final_df = pd.concat([hate_df_sampled, non_hate_df, school_df], ignore_index=True)
final_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)

# 7. 7:2:1 비율 분할
train_end = int(0.7 * len(final_df))
valid_end = int(0.9 * len(final_df))

train_df = final_df[:train_end]
valid_df = final_df[train_end:valid_end]
test_df = final_df[valid_end:]

# 8. 파일 저장
train_df.to_csv("/mnt/data/Train.csv", index=False)
valid_df.to_csv("/mnt/data/Valid.csv", index=False)
test_df.to_csv("/mnt/data/Test.csv", index=False)

"/mnt/data/Train.csv", "/mnt/data/Valid.csv", "/mnt/data/Test.csv"


In [8]:
import pandas as pd

# NSMC 데이터 로드 (ratings_test.txt)
nsmc_file_path = 'C:/Users/smhrd/test/ML/gitdata/nsmc-master/nsmc.txt'
nsmc_df = pd.read_csv(nsmc_file_path, sep='\t')

# NSMC의 부정적인 리뷰(0)와 긍정적인 리뷰(1)로 라벨링되어 있음
# (이미 부정/긍정이 0과 1로 분리되어 있음)
nsmc_df['label'] = nsmc_df['label'].apply(lambda x: 1 if x == 1 else 0)

# UnSmile 데이터 로드 (unsmile_train_v1.0.tsv, unsmile_valid_v1.0.tsv)
unsmile_train_file_path = 'C:/Users/smhrd/test/ML/gitdata/korean_unsmile_dataset-main/unsmile_train_v1.0.tsv'
unsmile_valid_file_path = 'C:/Users/smhrd/test/ML/gitdata/korean_unsmile_dataset-main/unsmile_valid_v1.0.tsv'

unsmile_train_df = pd.read_csv(unsmile_train_file_path, sep='\t')
unsmile_valid_df = pd.read_csv(unsmile_valid_file_path, sep='\t')

# UnSmile 데이터에서 혐오 표현과 악플/욕설 컬럼을 통합하여 label 생성
hate_columns = ["여성/가족", "남성", "성소수자", "인종/국적", "연령", "지역", "종교", "기타 혐오", "악플/욕설"]
unsmile_df = pd.concat([unsmile_train_df, unsmile_valid_df], ignore_index=True)

# 혐오 표현이 있으면 label을 1로 설정
unsmile_df["label"] = unsmile_df[hate_columns].sum(axis=1).clip(upper=1)

# 필요한 컬럼만 유지
unsmile_df = unsmile_df[["문장", "label"]]
unsmile_df = unsmile_df[unsmile_df["label"] == 1]  # 혐오 표현이 있는 데이터만 필터링

# UnSmile 데이터에서 혐오 표현과 악플/욕설 컬럼을 부정적인 리뷰로 처리
unsmile_df['label'] = 0  # 부정적인 리뷰로 처리
unsmile_df.rename(columns={"문장": "document"}, inplace=True)  # 컬럼 이름 변경

# NSMC와 UnSmile 데이터 합치기
final_df = pd.concat([nsmc_df[['document', 'label']], unsmile_df], axis=0, ignore_index=True)

# 텍스트 정제: NaN을 빈 문자열로 대체하고 양쪽 공백 제거
final_df['document'] = final_df['document'].fillna('').apply(lambda x: x.strip())

# 숫자만 있는 행을 제거
final_df = final_df[~final_df['document'].str.match(r'^\d+$')]  # 숫자만 있는 행 제거

# 빈 문자열인 데이터 제거
final_df = final_df[final_df['document'] != '']  # 빈 문자열인 데이터 제거

# 2글자 이하인 데이터 제거
final_df = final_df[final_df['document'].apply(lambda x: len(x) > 2)]  # 2글자 이하 데이터 제거

# 데이터 셔플링
final_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)

# 부정적인 리뷰(0)와 긍정적인 리뷰(1) 분리
negative_df = final_df[final_df['label'] == 0]
positive_df = final_df[final_df['label'] == 1]

# 부정적인 리뷰(0)를 긍정적인 리뷰(1)와 동일한 개수로 언더샘플링
negative_df_under = negative_df.sample(n=positive_df.shape[0], random_state=42)

# 긍정적인 리뷰(1)와 언더샘플링된 부정적인 리뷰(0) 합치기
final_df_balanced = pd.concat([positive_df, negative_df_under], axis=0, ignore_index=True)

# 데이터 셔플링
final_df_balanced = final_df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# 데이터 분할 전에 긍정/부정 개수 확인
positive_count = final_df_balanced[final_df_balanced['label'] == 1].shape[0]
negative_count = final_df_balanced[final_df_balanced['label'] == 0].shape[0]

print(f"긍정 리뷰 개수: {positive_count}")
print(f"부정 리뷰 개수: {negative_count}")

# 7:2:1 비율 분할
train_end = int(0.7 * len(final_df_balanced))
valid_end = int(0.9 * len(final_df_balanced))

train_df = final_df_balanced[:train_end]
valid_df = final_df_balanced[train_end:valid_end]
test_df = final_df_balanced[valid_end:]

# 파일 저장
train_df.to_csv("C:/Users/smhrd/test/ML/DataSet/NSMC,unsmile/Train.csv", index=False)
valid_df.to_csv("C:/Users/smhrd/test/ML/DataSet/NSMC,unsmile/Valid.csv", index=False)
test_df.to_csv("C:/Users/smhrd/test/ML/DataSet/NSMC,unsmile/Test.csv", index=False)


긍정 리뷰 개수: 98695
부정 리뷰 개수: 98695
